<a href="https://colab.research.google.com/github/ma4ypic4y/heavy-metal-pollution-prediction/blob/main/get_NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/10k_gen_europe.csv')

In [ ]:
df.describe()

### GEE API

In [8]:
import ee
import datetime

In [ ]:
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [ ]:
ndvi_val = []
for i in range(len(df)):
  
    lon = df['Longitude'][i]
    lat = df['Latitude'][i]
    
    point = (ee.Geometry.Point(lon, lat))
    l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')
  
    image = ee.Image(l8.filterBounds(point)
        .filterDate('2015-01-01', '2015-12-31')
        .sort('CLOUD_COVER')
        .first())
  
    nir = image.select('B5');
    red = image.select('B4');
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    val = (ndvi.reduceRegion(
                          reducer=ee.Reducer.mean(),
                          geometry=point,
                          scale=10))
    
    try:
        ndvi_val.append(val.getInfo().get('NDVI'))
    except Exception:
        ndvi_val.append(-10)
        continue
    print(i)

In [11]:
ser = pd.Series(ndvi_val, copy=False)

In [12]:
ser.to_csv('NEWoutNDVI_10k_water.csv')

In [ ]:
ser.describe()

In [17]:
df['ndvi'] = ser

In [18]:
df['Class'] = 2

### filter


In [21]:
df_out = df.query('ndvi>0.2')

## to GEOjson

In [14]:
pip install geojson

In [23]:
import pandas as pd
import geojson

def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["Longitude"],
                                                    X["Latitude"]
                                                    )),
                            properties=dict(class_=X["Class"])))
    df.apply(insert_features, axis=1)
    with open('test_NDVI.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)


data2geojson(df_out[['Latitude','Longitude','Class']])